# Portfolio Optimization using Deep Reinforcement Learning
----

## 3.0 Stock Selection
---
Auto Encoders are employed to select the less volatile stocks by choosing stocks with less reconstruction error. These are the stocks that are included in the portfolio

### 3.1 Import Relevant Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

In [2]:
from numpy import array
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense, Activation
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
from keras import regularizers, optimizers

from sklearn import preprocessing

### 3.2 Load the Data

In [3]:
# Load the close prices dataset
prices_data = pd.read_csv('./datasets/close_prices.csv')

In [4]:
df = prices_data.copy()

In [5]:
df = df.reset_index(drop=True).set_index(['date'])

In [6]:
df.head()

,BRITANNIA.NS,AXISBANK.NS,TATASTEEL.NS,HEROMOTOCO.NS,TATACONSUM.NS,HDFCLIFE.NS,GRASIM.NS,JSWSTEEL.NS,BAJAJ-AUTO.NS,NTPC.NS,...,BHARTIARTL.NS,NESTLEIND.NS,APOLLOHOSP.NS,BAJFINANCE.NS,INFY.NS,ONGC.NS,SUNPHARMA.NS,POWERGRID.NS,INDUSINDBK.NS,BAJAJFINSV.NS
date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,2387.000000,569.799988,70.064255,3810.000000,317.799988,399.799988,1165.166992,271.700012,3345.050049,147.791672,...,484.966522,790.159973,1216.000000,1760.000000,522.250000,195.699997,585.400024,113.484406,1655.949951,530.875000
2018-01-02,2371.125000,568.599976,69.668869,3784.350098,314.950012,398.000000,1149.529175,268.000000,3348.000000,149.375000,...,480.279999,792.304993,1212.000000,1739.699951,521.000000,197.500000,582.000000,113.512527,1647.000000,522.900024
2018-01-03,2354.975098,565.450012,70.264328,3764.399902,315.149994,398.299988,1144.798096,272.250000,3310.199951,150.250000,...,473.610687,793.500000,1203.949951,1738.349976,515.799988,197.399994,578.950012,113.878151,1650.000000,517.979980
2018-01-04,2344.449951,565.000000,72.736679,3759.949951,313.500000,401.950012,1170.844482,281.899994,3274.250000,148.250000,...,475.142822,790.219971,1188.449951,1758.250000,510.500000,200.000000,583.950012,114.103149,1652.500000,513.500000
2018-01-05,2339.750000,566.000000,74.018112,3758.699951,315.850006,412.000000,1214.172241,289.899994,3294.000000,148.541672,...,488.932068,788.400024,1199.000000,1821.000000,513.200012,200.949997,587.349976,113.653152,1703.000000,514.494995


### 3.3 Define Functions

In [7]:
def defineAutoencoder(num_stock, encoding_dim = 5, verbose=0):
    
    """
    Function for fitting an Autoencoder
    """

    # connect all layers
    input = Input(shape=(num_stock,))

    encoded = Dense(encoding_dim, kernel_regularizer=regularizers.l2(0.00001),name ='Encoder_Input')(input)

    decoded = Dense(num_stock, kernel_regularizer=regularizers.l2(0.00001), name ='Decoder_Input')(encoded)
    decoded = Activation("linear", name='Decoder_Activation_function')(decoded)

    # construct and compile AE model
    autoencoder = Model(inputs=input, outputs=decoded)
    adam = optimizers.Adam(learning_rate=0.0005)
    autoencoder.compile(optimizer=adam, loss='mean_squared_error')
    if verbose!= 0:
        autoencoder.summary()

    return autoencoder

In [8]:
def getReconstructionErrorsDF(df_pct_change, reconstructed_data):
    
    """
    Function for calculating the reconstruction Errors
    """
    array = []
    stocks_ranked = []
    num_columns = reconstructed_data.shape[1]
    for i in range(0, num_columns):
        diff = np.linalg.norm((df_pct_change.iloc[:, i] - reconstructed_data[:, i]))  # 2 norm difference
        array.append(float(diff))

    ranking = np.array(array).argsort()
    r = 1
    for stock_index in ranking:
        stocks_ranked.append([ r
                              ,stock_index
                              ,df_pct_change.iloc[:, stock_index].name
                              ,array[stock_index]
                              ])
        r = r + 1

    columns = ['ranking','stock_index', 'stock_name' ,'recreation_error']
    df = pd.DataFrame(stocks_ranked, columns=columns)
    df = df.set_index('stock_name')
    return df

### 3.4 Get the Percentage Change of the Close Prices

In [9]:
col_names = df.columns.to_list()

In [10]:
print(col_names)

['BRITANNIA.NS', 'AXISBANK.NS', 'TATASTEEL.NS', 'HEROMOTOCO.NS', 'TATACONSUM.NS', 'HDFCLIFE.NS', 'GRASIM.NS', 'JSWSTEEL.NS', 'BAJAJ-AUTO.NS', 'NTPC.NS', 'SBIN.NS', 'M&M.NS', 'HCLTECH.NS', 'SHRIRAMFIN.NS', 'ADANIENT.NS', 'DRREDDY.NS', 'ITC.NS', 'LTIM.NS', 'ADANIPORTS.NS', 'COALINDIA.NS', 'TITAN.NS', 'EICHERMOT.NS', 'ICICIBANK.NS', 'BPCL.NS', 'SBILIFE.NS', 'HDFCBANK.NS', 'LT.NS', 'HINDALCO.NS', 'CIPLA.NS', 'DIVISLAB.NS', 'ASIANPAINT.NS', 'TATAMOTORS.NS', 'TCS.NS', 'KOTAKBANK.NS', 'MARUTI.NS', 'TECHM.NS', 'HINDUNILVR.NS', 'RELIANCE.NS', 'WIPRO.NS', 'ULTRACEMCO.NS', 'BHARTIARTL.NS', 'NESTLEIND.NS', 'APOLLOHOSP.NS', 'BAJFINANCE.NS', 'INFY.NS', 'ONGC.NS', 'SUNPHARMA.NS', 'POWERGRID.NS', 'INDUSINDBK.NS', 'BAJAJFINSV.NS']


In [11]:
df_pct_change = df.pct_change(1).astype(float)
df_pct_change = df_pct_change.replace([np.inf, -np.inf], np.nan)
df_pct_change = df_pct_change.fillna(method='bfill')

# the percentage change function will make the first two rows equal to nan
df_pct_change = df_pct_change.tail(len(df_pct_change) - 2)

/var/folders/3z/6b2hp23s5sg2j9x4jt4hyh_80000gn/T/ipykernel_165/2395198504.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_pct_change = df_pct_change.fillna(method='bfill')


In [12]:
if df_pct_change.isnull().values.any():
    print("Warning: NaN values detected in df_pct_change. Please handle them before calculating reconstruction errors.")


In [13]:
df_pct_change.shape

(1519, 50)

In [14]:
# remove columns where there is no change over a longer time period
df_pct_change = df_pct_change[df_pct_change.columns[((df_pct_change == 0).mean() <= 0.05)]]

In [15]:
df_pct_change.head()

,BRITANNIA.NS,AXISBANK.NS,TATASTEEL.NS,HEROMOTOCO.NS,TATACONSUM.NS,HDFCLIFE.NS,GRASIM.NS,JSWSTEEL.NS,BAJAJ-AUTO.NS,NTPC.NS,...,BHARTIARTL.NS,NESTLEIND.NS,APOLLOHOSP.NS,BAJFINANCE.NS,INFY.NS,ONGC.NS,SUNPHARMA.NS,POWERGRID.NS,INDUSINDBK.NS,BAJAJFINSV.NS
date,,,,,,,,,,,,,,,,,,,,,
2018-01-03,-0.006811,-0.005540,0.008547,-0.005272,0.000635,0.000754,-0.004116,0.015858,-0.011290,0.005858,...,-0.013886,0.001508,-0.006642,-0.000776,-0.009981,-0.000506,-0.005241,0.003221,0.001821,-0.009409
2018-01-04,-0.004469,-0.000796,0.035186,-0.001182,-0.005236,0.009164,0.022752,0.035445,-0.010860,-0.013311,...,0.003235,-0.004134,-0.012874,0.011448,-0.010275,0.013171,0.008636,0.001976,0.001515,-0.008649
2018-01-05,-0.002005,0.001770,0.017617,-0.000332,0.007496,0.025003,0.037006,0.028379,0.006032,0.001967,...,0.029021,-0.002303,0.008877,0.035689,0.005289,0.004750,0.005822,-0.003944,0.030560,0.001938
2018-01-08,0.006731,0.003534,0.001223,0.008859,0.010606,0.039563,-0.002871,-0.001380,0.000820,0.001402,...,-0.015392,0.010724,0.008340,0.012081,0.012276,-0.005225,0.029369,0.005939,0.012331,0.021001
2018-01-09,0.000499,0.008803,0.003857,0.003376,0.001410,0.045179,0.023365,0.000000,-0.003701,-0.000560,...,-0.024525,0.000038,-0.015095,0.000000,0.005679,-0.004502,-0.010916,-0.009102,0.003451,-0.002018


### 3.5 Construct the Autoencoder

In [16]:
# define the input parameters
hidden_layers = 5
batch_size = 500
epochs = 500
stock_selection_number = 20
num_stock = df_pct_change.shape[1]
verbose = 1

In [17]:
print('-' * 20 + 'Step 1 : Returns vs. recreation error (recreation_error)')
print('-' * 25 + 'Transform dataset with MinMax Scaler')

--------------------Step 1 : Returns vs. recreation error (recreation_error)
-------------------------Transform dataset with MinMax Scaler


In [18]:
# Normalize the data
df_scaler = preprocessing.MinMaxScaler()
df_pct_change_normalised = df_scaler.fit_transform(df_pct_change)

In [19]:
# define autoencoder
print('-' * 25 + 'Define autoencoder model')
num_stock = len(df_pct_change.columns)
autoencoder = defineAutoencoder(num_stock=num_stock, encoding_dim=hidden_layers, verbose=verbose)
#plot_model(autoencoder, to_file='img/model_autoencoder_1.png', show_shapes=True,
#           show_layer_names=True)

-------------------------Define autoencoder model


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Encoder_Input (Dense)           │ (None, 5)              │           255 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Decoder_Input (Dense)           │ (None, 50)             │           300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Decoder_Activation_function     │ (None, 50)             │             0 │
│ (Activation)                    │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 555 (2.17 KB)

 Trainable params: 555 (2.17 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# train autoencoder
print('-' * 25 + 'Train autoencoder model')
autoencoder.fit(df_pct_change_normalised, df_pct_change_normalised, shuffle=False, epochs=epochs,
                batch_size=batch_size,
                verbose=verbose)

-------------------------Train autoencoder model
Epoch 1/500


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3100  
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 726us/step - loss: 0.2920
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step - loss: 0.2756
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step - loss: 0.2606
Epoch 5/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step - loss: 0.2470
Epoch 6/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 596us/step - loss: 0.2347
Epoch 7/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2235
Epoch 8/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - loss: 0.2132
Epoch 9/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step - loss: 0.2038
Epoch 10/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 639us/step - loss: 0.1950
Epoch 11/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step - loss: 0.1868
Epoch 12/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 780us/step - loss: 0.1791
Epoch 13/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step - loss: 0.1720
Epoch 14/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1652
Epoch 15/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 676us/step - loss: 0.1589
Epoch 16/500
4/4 ━━━

In [21]:
# predict autoencoder
print('-' * 25 + 'Predict autoencoder model')
reconstruct = autoencoder.predict(df_pct_change_normalised)

-------------------------Predict autoencoder model
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 360us/step


In [22]:
# Inverse transform dataset with MinMax Scaler
print('-' * 25 + 'Inverse transform dataset with MinMax Scaler')
reconstruct_real = df_scaler.inverse_transform(reconstruct)
df_reconstruct_real = pd.DataFrame(data=reconstruct_real, columns=df_pct_change.columns)

-------------------------Inverse transform dataset with MinMax Scaler


In [23]:
print('-' * 25 + 'Calculate L2 norm as reconstruction loss metric')
df_recreation_error = getReconstructionErrorsDF(df_pct_change=df_pct_change,
                                                reconstructed_data=reconstruct_real)

-------------------------Calculate L2 norm as reconstruction loss metric


In [24]:
df_recreation_error

,ranking,stock_index,recreation_error
stock_name,,,
NESTLEIND.NS,1,41,0.412935
HDFCBANK.NS,2,25,0.463144
HINDUNILVR.NS,3,36,0.488788
TCS.NS,4,32,0.489211
KOTAKBANK.NS,5,33,0.499979
HCLTECH.NS,6,12,0.520578
WIPRO.NS,7,38,0.530612
ASIANPAINT.NS,8,30,0.531486
MARUTI.NS,9,34,0.534654


In [25]:
filtered_stocks = df_recreation_error.head(stock_selection_number).index

In [26]:
filtered_stocks

Index(['NESTLEIND.NS', 'HDFCBANK.NS', 'HINDUNILVR.NS', 'TCS.NS',
       'KOTAKBANK.NS', 'HCLTECH.NS', 'WIPRO.NS', 'ASIANPAINT.NS', 'MARUTI.NS',
       'TITAN.NS', 'NTPC.NS', 'BAJAJ-AUTO.NS', 'RELIANCE.NS', 'POWERGRID.NS',
       'LT.NS', 'ICICIBANK.NS', 'GRASIM.NS', 'ITC.NS', 'SBILIFE.NS',
       'HDFCLIFE.NS'],
      dtype='object', name='stock_name')

In [27]:
# store the list of selected stocks
%store filtered_stocks

Stored 'filtered_stocks' (Index)


In [28]:
pip install pickleshare

Note: you may need to restart the kernel to use updated packages.


In [29]:
%store

Stored variables and their in-db values:
df_close_full_stocks             ->             date  BRITANNIA.NS  AXISBANK.NS  TATAS
filtered_stocks                  -> Index(['NESTLEIND.NS', 'HDFCBANK.NS', 'HINDUNILVR.
